# Подготовка исходного датасета

In [1]:
# Эагрузка файла, предварительно скачанного с PubChem 
# по ссылке https://pubchem.ncbi.nlm.nih.gov/#query=PubChem%20Compound%20TOC%3A%20Toxicity
# Ссылка - это поиск по фразе 'PubChem Compound TOC: Toxicity', 
# выдает (предположительно) вещества, где в описании секция Toxicity не пустая
import pandas as pd
df = pd.read_csv(r'../data/raw/pubchem/PubChem_compound_cache_hcIgmJxu-dLO_HHl8504zfcrnUtTnzWLT64ux1S_PMZUpgA.csv')

In [2]:
# В моем файле дефект - лишний пробел в начале, поэтому 
# переименуем ' cid' в 'cid'
df = df.rename(columns={' cid': 'cid'})

In [104]:
# создадим пустой датафрейм для приема запрошенных данных
toxicity_data = pd.DataFrame(
    {
        "cid": pd.Series(dtype="int64"),
        "status_code": pd.Series(dtype="int16"),
        "toxicity_data_info": pd.Series(dtype="object"),
        "toxicity_data_value": pd.Series(dtype="object"),
    }
)

# заготовим пустые строки с номерами CID 
toxicity_data['cid'] = df.cid   

# Загрузка через API

### Версия загрузки 0.1.1 (устарело)

In [315]:
import random
import time
import logging
# чтобы это сработало, предварительно выполнить в корне проекта
# $ pip install .
from datasets.pubchem import pubchem as pc 
import pandas as pd

In [ ]:
toxicity_data = pd.read_csv(r'../data/raw/pubchem/toxicity_data.csv')

start = 0
stop = 118040 # это максимальный номер в скачанном файле

# version 0.1.1 - если было 404 по версии 0.1.0, то повторный запрос всей секции Toxicity и сохранение всего отвеа

for index, row in toxicity_data.loc[start:stop].iterrows():
    if row['status_code'] in [200]:
        continue
    else:
        try:
            res = pc.get_toxicity_data(row['cid'])
            if res.is_success:
                toxicity_data.at[index, 'status_code'] = int(res.status_code)
                toxicity_data.at[index, 'toxicity_data_info'] = res.json()
                toxicity_data.at[index, 'toxicity_data_value'] = [pc.get_toxicity_data_value(res.json())]
                toxicity_data.at[index, 'version'] = '0.1.0'
            else:
                res = pc.get_toxicity_data(row['cid'], full_toxicity_data=True)
                if res.is_success:
                    toxicity_data.at[index, 'status_code'] = int(res.status_code)
                    toxicity_data.at[index, 'toxicity_data_info'] = res.json()
                    # TODO: нужна функция разбора всей секции Toxicity и сохранения всех вхождений LD/LC
                    # toxicity_data.at[index, 'toxicity_data_value'] = [ ]
                    toxicity_data.at[index, 'version'] = '0.1.1'

            print(f"{index}/{stop}: cid={row['cid']}, {res.status_code}")
        except Exception as e:
            toxicity_data.at[index, 'status_code'] = 0
            toxicity_data.at[index, 'toxicity_data_info'] = None
            toxicity_data.at[index, 'toxicity_data_value'] = None
            logging.error(f"{index}/{stop}: cid={row['cid']}: {e}")
    if index % 10000 == 0:
        toxicity_data.to_csv(f'../data/raw/pubchem/toxicity_data_index-0.1.0-{index}.csv', index=False)
    time.sleep(random.random()/4+0.2)

### Отладка загрузки

In [45]:
import pandas as pd
td = pd.read_csv("../data/raw/pubchem/toxicity_data_index-0.1.0-10000.csv")

In [46]:
import json

def find_string_elements(json_data, patterns=["ld50", "ldlo", "lc50", "td50", "bcf"]):
  """
  Находит все элементы с ключом "String", значения которых содержат "LD50" или "LC50".

  Args:
    json_data: JSON-данные для анализа.
    patterns: Список шаблонов для поиска. По умолчанию: ["ld50", "ldlo", "lc50", "td50", "bcf"]
  Returns:
    list: Список элементов, удовлетворяющих условию.
  """
  results = []

  def search(data):
    if isinstance(data, dict):
      for key, value in data.items():
        if key == "String" and any(pattern in value.lower() for pattern in patterns):
          results.append(value)
        elif isinstance(value, (dict, list)):
          search(value)
    elif isinstance(data, list):
      for item in data:
        search(item)

  search(json_data)
  return results


In [70]:
import pandas as pd
from datasets.pubchem import pubchem as pc 
r = pc.get_toxicity_data(935, full_toxicity_data=True)

In [71]:
json_data = json.loads(r.text)  
patterns = ["ld50", "ldlo", "lc50", "td50", "bcf"]
results = find_string_elements(r.json(), patterns)

for result in results:
  print(result)

In [ ]:
r.json()